In [ ]:
#Librerias
import pandas as pd
import xgboost as xgb

In [ ]:
#Carga muestra
path_muestra = "muestra"
# Read the parquet file (remove undefined 'nam' argument)
df = pd.read_parquet(path_muestra)
print(df.head())

In [ ]:
#Carga funcion y modelo
import predict_function
model_occ = xgb.Booster()  # init model
path_model = './model.json'
model_occ.load_model(path_model)

In [ ]:
caso = 1

In [ ]:
predicciones = predict_function.pred_funct(
    model_occ,
    **df.loc[caso, model_occ.feature_names].to_dict()
)
predicciones

In [ ]:
results = []
for i in range(len(df)):
    pred = predict_function.pred_funct(
        model_occ,
        **df.loc[i, model_occ.feature_names].to_dict()
    )
    results.append(pred)

In [ ]:
results

In [ ]:
results_df = pd.DataFrame(results, df.index)
results_df

In [ ]:


df = pd.concat([df, results_df], axis=1)
df

In [ ]:
predcciones  = df[['raw_output', 'prob_sigmoid']]


In [ ]:
predcciones

In [ ]:
# Ejecutar predicción para todos los casos y guardar resultados
results = []
for idx in df.index:
    features = df.loc[idx, model_occ.feature_names].to_dict()
    res = predict_function.pred_funct(model_occ, **features)
    results.append(res)

preds_df = pd.DataFrame(results, index=df.index)

# Añadir las columnas de predicción al dataframe original
df = pd.concat([df, preds_df], axis=1)
df 
# Guardar resultado completo y mostrar primeras filas
predicciones_all = df[['raw_output', 'prob_sigmoid']]
predicciones_all

In [ ]:
predict_function

In [ ]:
#Cargar librerias
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
#Definir ruta del modelo
model_path = 'model.json'

In [ ]:
#Cargar el modelo
model = xgb.Booster()
model.load_model(model_path)

In [ ]:
#Cantidad de arboles en el modelo
print("Cantidad de arboles en el modelo:", model.num_boosted_rounds())

In [ ]:
# ensure python graphviz package is installed (needed by xgboost.plot_tree)
%pip install graphviz

import shutil

fig, ax = plt.subplots(figsize=(30, 10))

# xgboost.plot_tree requires the Graphviz 'dot' executable. If it's missing, fall back to feature importance.
if shutil.which("dot") is None:
	print("Graphviz 'dot' executable not found. Falling back to plotting feature importance.")
	print("To enable tree plotting, install Graphviz on your system and restart the kernel.")
	xgb.plot_importance(model, ax=ax, max_num_features=30)
else:
	try:
		xgb.plot_tree(model, num_trees=0, ax=ax)
	except ImportError as e:
		print("ImportError while plotting tree:", e)
		print("Falling back to plotting feature importance.")
		xgb.plot_importance(model, ax=ax, max_num_features=30)

plt.tight_layout()
plt.show()

In [ ]:
xgb.plot_importance(model, max_num_features=30)
plt.tight_layout()
plt.show()

In [ ]:
num_trees = model.num_boosted_rounds()
print("Número de árboles en el modelo:", num_trees)

In [ ]:
for i in range(num_trees):
    fig, ax = plt.subplots(figsize=(30, 10))
    xgb.plot_tree(model, num_trees=i, ax=ax)
    plt.tight_layout()
    plt.show()

    fname = f'tree_{i}.png'
    fig.savefig(fname)
    print(f'Saved tree {i} as {fname}')